#Light

Используя шаблон ноутбука занятия сделайте парсинг базы headhunter, исходя из описания функций. Добавьте в конечную базу готовность к переезду и командировкам (2 дополнительных столбца)

In [ ]:
%matplotlib inline

import re

import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from google.colab import files
from google.colab import drive
from datetime import datetime

In [ ]:
hh_path = '/content/drive/My Drive/Assets/lab2/hh.csv'

In [ ]:
fixed_df = pd.read_csv(hh_path)

cols = fixed_df.columns.tolist()

i=0
for j in range(len(cols)):
    for i in range(fixed_df.shape[0]):
        if type(fixed_df.values[i][j])!=float:
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\xa0","")
            fixed_df.values[i][j] = fixed_df.values[i][j].replace("\n"," ")

fixed_df.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последеняя/нынешняя должность,Образование и ВУЗ,Авто
0,"Женщина, 21год, родилась 26 марта 1998",30000 руб.,Специалист по поддержке пользователей,"Тимашевск, готова к переезду, готова к команди...","проектная работа, стажировка, частичная занят...","удаленная работа, гибкий график, полный день,...",Опыт работы 4 месяца Октябрь2018 — январь2019...,Медицинская сестра палатная,"Среднее специальное образование 2018 ГБПОУ ""Ей...",NaN
1,"Мужчина, 23года, родился 17 января 1996",NaN,Программист,"Воронеж, не готов к переезду, не готов к коман...",полная занятость,полный день,Опыт работы 2 года 9 месяцев Октябрь2016 — ию...,Сервисный инженер,Образование Среднее специальное образование,Имеется собственный автомобиль
2,Мужчина,NaN,Веб-программист,"Мытищи, не готов к переезду, не готов к команд...",полная занятость,полный день,Опыт работы 3 года 1 месяц Январь2019 — июль2...,Web-разработчик,Неоконченное высшее образование 2019 Московски...,NaN
3,"Мужчина, 35лет, родился 24 октября 1983",50000 руб.,Аналитик-консультант 1С,"Нижний Новгород, не готов к переезду, готов к ...",полная занятость,"гибкий график, полный день, сменный график",Опыт работы 13 лет 1 месяц Март2016 — понасто...,Специалист по планированию производства,Высшее образование 2006 Волго-Вятская академия...,NaN
4,"Мужчина, 28лет, родился 3 ноября 1990",45000 руб.,Электромонтер связи,"Москва, м. Отрадное, готов к переезду, готов к...",полная занятость,полный день,Опыт работы 7 лет 8 месяцев Декабрь2018 — июн...,Инженер,Среднее специальное образование 2011 Колледж с...,NaN


In [ ]:
def getParameterSexAge(arg):
  out = 0
  if "М" in arg:
    sex='М'
  elif "Ж" in arg:
    sex = 'Ж'
  else:
    sex = "Не указан"

  year_tec = datetime.now().year
  if (len(arg) > 7):
    out = year_tec - int(re.findall(r'\d{4}', arg)[0])

  return sex, str(out)

def getParameterSalary(arg):
    arg=str(arg)

    if(arg == 'nan'):
      return "Не указано"

    salaryStr = int(re.sub(r'[^0-9]+', '', arg))
    curr_ = re.sub('[0-9]','',arg).replace('.','')

    if 'USD' in curr_ :   salaryStr = float(salaryStr)*65
    elif 'KZT' in curr_:  salaryStr = float(salaryStr)*0.17
    elif 'грн' in curr_:  salaryStr = float(salaryStr)*2.6
    num = salaryStr
    return salaryStr

def getParameterCity(arg):
  millionCities = "Новосибирск Екатеринбург Нижний Новгород Казань Челябинск Омск Самара Ростов-на-Дону Уфа Красноярск Пермь Воронеж Волгоград Сочи"
  sarg=arg.split(',')
  item = sarg[0]
  item= item.strip()
  if item == "Москва":          return "Москва"
  if item == "Санкт-Петербург": return "Санкт-Петербург"
  if item in millionCities:     return item
  else: return "малый или за пределами России город"

def getParameterEmployment(arg):
  out= "Не указано"
  if ("стажировка" in arg)|("volunteer" in arg)|("work placement" in arg): return 'стажировка'
  elif ("частичная занятость" in arg)|("part" in arg):  return 'частичная занятость'
  elif ("проектная работа" in arg)|("project" in arg):  return 'проектная работа'
  elif ("полная занятость" in arg)|("full" in arg):     return 'полная занятость'
  return out

def getParameterSchedule(arg):
  out = "Не указано"
  if ("гибкий график" in arg)|("flexible" in arg):    return 'гибкий график'
  if ("полный день" in arg)|("full" in arg):      return 'полный день'
  if ("сменный график" in arg)|("shift" in arg):   return 'сменный график'
  if ("удаленная работа" in arg)|("remote" in arg): return 'удаленная работа'
  return out

def getParameterEducation(arg):
  out = "Не указано"
  arg = arg.lower()
  if ("высшее" in arg)|("higher education"  in arg): return 'высшее'
  if ("cреднее специальное"  in arg) | ("special"  in arg) | ("колледж"  in arg) | ("college"  in arg) : return 'среднее специальное'
  if ("неоконченное высшее"  in arg) | ("incomplete"  in arg): return 'неоконченное высшее'
  if ("среднее образование"  in arg) | ("secondary education"  in arg): return 'среднее образование'
  return out

def getParameterExperience(arg):
  sarg=arg.split(" ", 5)
  years = 0
  months = 0
  if len(sarg)!=6: sarg.append(0);sarg.append('');sarg.append(0);sarg.append('')
  try:
    years = int(sarg[2])
  except ValueError:
    years = 0

  try:
    months = int(sarg[4])
  except ValueError:
    months = 0
  return 12*years+months

def getParameterIsReadyToMove(arg):
  ready_f = 'готова к переезду'
  ready_m = 'готов к переезду'

  not_ready_f = f'не ${ready_f}'
  not_ready_m = f'не ${ready_m}'

  arg = arg.lower()

  if(not_ready_f in arg):
    return not_ready_f
  elif(not_ready_m in arg):
    return not_ready_m
  elif(ready_f in arg):
    return ready_f
  elif(ready_m in arg):
    return ready_m
  else:
    return 'Не указано'

def getParameterIsReadyForPromotions(arg):
  ready_f = 'готова к командировкам'
  ready_m = 'готов к командировкам'

  not_ready_f = f'не ${ready_f}'
  not_ready_m = f'не ${ready_m}'

  arg = arg.lower()

  if(not_ready_f in arg):
    return not_ready_f
  elif(not_ready_m in arg):
    return not_ready_m
  elif(ready_f in arg):
    return ready_f
  elif(ready_m in arg):
    return ready_m
  else:
    return 'Не указано'



In [ ]:
modified_df = fixed_df.copy()

modified_df = modified_df.rename(columns={'Опыт (двойное нажатие для полной версии)': 'Опыт'})

sex_and_age = modified_df['Пол, возраст'].map(getParameterSexAge)

modified_df.insert(0,'Пол', [sex_and_age[i][0] for i in range(len(sex_and_age))])
modified_df.insert(1,'Возраст', [sex_and_age[i][1] for i in range(len(sex_and_age))])
modified_df = modified_df.drop(columns=['Пол, возраст'])

readyToMove = modified_df['Город'].map(getParameterIsReadyToMove)
readyForPromotions = modified_df['Город'].map(getParameterIsReadyForPromotions)

modified_df.insert(5, 'Готовность к переезду', readyToMove)
modified_df.insert(6, 'Готовность к комондировкам', readyForPromotions)

modified_df['ЗП'] = modified_df['ЗП'].map(getParameterSalary)
modified_df['Занятость'] = modified_df['Занятость'].map(getParameterEmployment)
modified_df['Город'] = modified_df['Город'].map(getParameterCity)
modified_df['График'] = modified_df['График'].map(getParameterSchedule)
modified_df['Опыт'] = modified_df['Опыт'].map(getParameterExperience)
modified_df['Образование и ВУЗ'] = modified_df['Образование и ВУЗ'].map(getParameterEducation)
modified_df['Авто'] = modified_df['Авто'].fillna('Нет авто')


modified_df.to_csv('/content/drive/My Drive/Assets/lab2/modified_hh.csv', index=False)

modified_df.head()

,Пол,Возраст,ЗП,Ищет работу на должность:,Город,Готовность к переезду,Готовность к комондировкам,Занятость,График,Опыт,Последеняя/нынешняя должность,Образование и ВУЗ,Авто
0,Ж,27,30000,Специалист по поддержке пользователей,малый или за пределами России город,готова к переезду,готова к командировкам,стажировка,гибкий график,48,Медицинская сестра палатная,среднее специальное,Нет авто
1,М,29,Не указано,Программист,Воронеж,готов к переезду,готов к командировкам,полная занятость,полный день,33,Сервисный инженер,Не указано,Имеется собственный автомобиль
2,М,0,Не указано,Веб-программист,малый или за пределами России город,готов к переезду,готов к командировкам,полная занятость,полный день,37,Web-разработчик,высшее,Нет авто
3,М,42,50000,Аналитик-консультант 1С,Нижний Новгород,готов к переезду,готов к командировкам,полная занятость,гибкий график,157,Специалист по планированию производства,высшее,Нет авто
4,М,35,45000,Электромонтер связи,Москва,готов к переезду,готов к командировкам,полная занятость,полный день,92,Инженер,среднее специальное,Нет авто


Pro

Возьмите базу квартир (moscow.csv), проведите парсинг и очистку данных:
Уберите строки “Для заметок”
Уберите столбцы “ГРМ” и “Примечания”
Для столбца “Балкон” добавьте категорию “Отсутствует” (вместо NaN)
Значения столбца “Бонус агенту” переведите в тысячи рублей,а пустые значения заполните нулями
В столбец “Площадь” запишите общую площадь всех помещений
Для столбца “Санузел” заполните отсутствующие значения строкой “Нет данных”
Столбец “Цена” переведите в тысячи рублей


In [ ]:
%matplotlib inline

import re

import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from google.colab import files
from google.colab import drive
from datetime import datetime

path = '/content/drive/MyDrive/Assets/lab2/moscow.csv'

csv = pd.read_csv(path, sep=';')
csv.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, руб.",ГРМ,Бонус агенту,Дата,Кол-во дней в экспозиции,Источник,Примечание
0,1,Шелепиха м.,12п,35/37 М,NaN,2,64.1/23/20,19500000.0,NaN,NaN,25.07.2019,1.0,Привлекательные,Собственность оформлена! Ключи на руках! Опера...
1,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,38.4/17/9,4800000.0,NaN,NaN,25.07.2019,2.0,Привлекательные,Зеленоград к.1820 однокомнатная квартира общ.3...
3,Для заметок:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,38.2/19.9/9.9,7990000.0,NaN,NaN,25.07.2019,4.0,Привлекательные,"Уютная, светлая квартира, застекленный балкон ..."


In [ ]:
def getParameterBalcony(arg):
  return "Отсутствует" if pd.isna(arg) or arg == '-' else arg

def getParameterAgentBonus(price, bonus):
  price = float(str(price).strip())

  if pd.isna(bonus):
      return 0.0

  bonus = str(bonus).strip().replace(' ', '')

  if '%' in bonus:
      return price * float(bonus.replace('%', '')) / 100 / 1000
  else:
      return float(bonus.replace('руб.', '').strip()) / 1000

def getArea(arg):
  if(pd.isna(arg)):
    return 0
  if('/' not in arg):
    return 0
  if('?' in arg):
    return 0

  arg = arg.split('/')
  out = sum([float(i.replace(',', '.')) for i in arg])
  return out

modified_csv = csv.copy()
modified_csv = modified_csv.drop(columns = ['Примечание', 'ГРМ'])

modified_csv['Бонус агенту'] = modified_csv.apply(lambda x: getParameterAgentBonus(x['Цена, руб.'], x['Бонус агенту']), axis=1)
modified_csv['Балкон'] = modified_csv['Балкон'].map(getParameterBalcony)
modified_csv['Санузел'] = modified_csv['Санузел'].fillna('Нет данных')
modified_csv['Площадь'] = modified_csv['Площадь'].map(getArea)
modified_csv['Цена, руб.'] = modified_csv['Цена, руб.'].apply(lambda x: x / 1000 if pd.notna(x) else x)

modified_csv = modified_csv.rename(columns={'Цена, руб.': 'Цена, тыс. руб.'})

modified_df.to_csv('/content/drive/My Drive/Assets/lab2/modified_moscow.csv', index=False)

modified_csv.head()

,Комнат,Метро / ЖД станции,От станции,Дом,Балкон,Санузел,Площадь,"Цена, тыс. руб.",Бонус агенту,Дата,Кол-во дней в экспозиции,Источник
0,1,Шелепиха м.,12п,35/37 М,Отсутствует,2,107.1,19500.0,0.0,25.07.2019,1.0,Привлекательные
1,Для заметок:,NaN,NaN,NaN,Отсутствует,Нет данных,0.0,NaN,0.0,NaN,NaN,NaN
2,1,Пятницкое шоссе м.,30т,11/14 П,Л,Р,64.4,4800.0,0.0,25.07.2019,2.0,Привлекательные
3,Для заметок:,NaN,NaN,NaN,Отсутствует,Нет данных,0.0,NaN,0.0,NaN,NaN,NaN
4,1,Планерная м.,5т,11/22 П,Б,С,68.0,7990.0,0.0,25.07.2019,4.0,Привлекательные
